In [1]:
"""
The purpose of this Jupyter notebook is to process the Dharmacon pooled
Genome 1 and Genome 2 subset of the VACV screen.
"""

'\nThe purpose of this Jupyter notebook is to process the Dharmacon pooled\nGenome 1 and Genome 2 subset of the VACV screen.\n'

In [1]:
import os
import math

import pandas as pd
from biotite.sequence.io import fasta

#### Extraction of Dharmacon Pooled Genome 1 & 2 Screening Plates Subset

In [32]:
path_to_entire_vacv_screen = (
    "/Users/jacobanter/Documents/Code/VACV_screen/VacciniaReport_"
    "20170223-0958_ZScored_conc_and_NaN_adjusted.csv"
)

entire_vacv_screen_df = pd.read_csv(
    path_to_entire_vacv_screen,
    sep="\t"
)

/var/folders/j1/64kbg_f11z97kx1dw__420vh0000gn/T/ipykernel_3966/4203157425.py:6: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  entire_vacv_screen_df = pd.read_csv(


In [33]:
# Now, extract the subset of interest, i.e. the Dharmacon pooled genome
# 1 and 2 subset with screening plates and no checkerboard plates
# Note that the `query()` method can be used instead of square brackets
# in order to avoid entering the DataFrame name multiple times
dp_g1_g2_subset_df = entire_vacv_screen_df.query(
    "(Experiment == 'VACCINIA-DP-G1' or Experiment == 'VACCINIA-DP-G2') "
    "and "
    "PLATE_TYPE == 'ScreeningPlate'"
)

In [34]:
assert len(dp_g1_g2_subset_df) == 43776, (
    "Something went wrong while extracting the subset from the screen!"
)

In [35]:
# Save the Dharmacon pooled subset to a TSV file
dp_g1_g2_subset_df.to_csv(
    "Dharmacon_pooled_G1_G2_screening_plates_subset.tsv",
    sep="\t",
    index=False
)

#### Determining Unique Gene IDs

In [20]:
# Load the Dharmacon pooled subset into a DataFrame
path_to_dharmacon_pooled_subset = (
    "Dharmacon_pooled_G1_G2_screening_plates_subset.tsv"
)

dp_g1_g2_subset_df = pd.read_csv(
    path_to_dharmacon_pooled_subset,
    sep="\t"
)

In [21]:
# Determine the unique values in the "ID_manufacturer" column to query
# NCBI Entrez with
unique_ids = dp_g1_g2_subset_df["ID_manufacturer"].unique()

In [22]:
print(f"There are {len(unique_ids):,} unique gene ids.")

There are 18,042 unique gene ids.


In [23]:
for unique_id in unique_ids:
    try:
        int(unique_id)
    except ValueError:
        print(unique_id)

Not available


In [24]:
# In order to download information from the NCBI Entrez gene database,
# these 18,042 gene ids are put into a text file with one gene ID per
# line
# Bear in mind that in the context of working with files, the `with`
# context manager is preferred as it automatically takes care of closing
# files, even in case of errors/exceptions
with open("gene_ids.txt", "w") as f:
    for i, unique_id in enumerate(unique_ids):
        # Note that the unique IDs may comprise some special values,
        # such as "Not available"
        # These need to be omitted by filtering for integers/integer
        # sequences
        try:
            int(unique_id)
        except ValueError:
            continue

        if i != (len(unique_ids) - 1):
            f.write(unique_id + "\n")
        else:
            f.write(unique_id)

#### Processing NCBI Entrez Gene Database Files from the FTP File Server

In [ ]:
# As it turns out, downloading files directly from the NCBI Entrez FTP
# file server is far more convenient than using the `datasets` CLI
# Thus, two files have been downloaded, namely `gene_info.gz` as well as
# `gene_history.gz`
# The latter is required in order to identify withdrawn or replaced gene
# IDs
# However, both files contain entries for all species, which is why they
# have to be filtered to include only human entries (taxonomic ID 9606)

# First, process `gene_info`
gene_info_df = pd.read_csv(
    "gene_info",
    sep="\t"
)

/var/folders/j1/64kbg_f11z97kx1dw__420vh0000gn/T/ipykernel_1194/2071511180.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  gene_info_df = pd.read_csv(


In [9]:
gene_info_df = gene_info_df[
    gene_info_df["#tax_id"] == 9606
]

In [12]:
gene_info_df.to_csv(
    "gene_info_human_9606.tsv",
    sep="\t",
    index=False
)

In [13]:
gene_history_df = pd.read_csv(
    "gene_history",
    sep="\t"
)

In [14]:
gene_history_df = gene_history_df[
    gene_history_df["#tax_id"] == 9606
]

In [15]:
gene_history_df.to_csv(
    "gene_history_human_9606.tsv",
    sep="\t",
    index=False
)

In [16]:
# Additionally, two other files have been downloaded; they are required
# for the mapping of genes to UniProt accessions, i.e. proteins
# These two files are `gene2accession.gz` and
# `gene_refseq_uniprotkb_collab.gz`
# As for the two previous files, they have to be filtered to include
# only human entries (taxonomic ID 9606)
gene_refseq_uniprotkb_collab_df = pd.read_csv(
    "gene_refseq_uniprotkb_collab",
    sep="\t"
)

In [19]:
gene_refseq_uniprotkb_collab_df = gene_refseq_uniprotkb_collab_df[
    gene_refseq_uniprotkb_collab_df["NCBI_tax_id"] == 9606
]

In [21]:
gene_refseq_uniprotkb_collab_df.to_csv(
    "gene_refseq_uniprotkb_collab_human_9606.tsv",
    sep="\t",
    index=False
)

In [22]:
gene2accession_df = pd.read_csv(
    "gene2accession",
    sep="\t"
)

/var/folders/j1/64kbg_f11z97kx1dw__420vh0000gn/T/ipykernel_1194/1663381539.py:1: DtypeWarning: Columns (6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  gene2accession_df = pd.read_csv(


In [24]:
gene2accession_df = gene2accession_df[
    gene2accession_df["#tax_id"] == 9606
]

In [26]:
gene2accession_df.to_csv(
    "gene2accession_human_9606.tsv",
    sep="\t",
    index=False
)

#### Inserting Columns into the DataFrame

In [2]:
dp_g1_g2_subset_df = pd.read_csv(
    "Dharmacon_pooled_G1_G2_screening_plates_subset.tsv",
    sep="\t"
)

In [3]:
# The following columns are supposed to be inserted into the DataFrame:
# `Gene_type`, which, as its name already suggests, indicates the type
# of the respecticve gene
# `UniProt_IDs`, which stores the UniProt ID(s) associated with the
# respective gene
# `Withdrawn_by_NCBI`, which, as its name already implied, indicates
# whether the gene ID is still valid or not
# They are supposed to be inserted immediately after the
# "Name_alternatives" column
# Thus, its index has to be determined
columns_list = dp_g1_g2_subset_df.columns.to_list()

insertion_index = columns_list.index("Name_alternatives") + 1

dp_g1_g2_subset_df.insert(
    loc=insertion_index,
    column="Gene_type",
    value="Value not set"
)

insertion_index += 1

dp_g1_g2_subset_df.insert(
    loc=insertion_index,
    column="UniProt_IDs",
    value="Value not set"
)

insertion_index += 1

dp_g1_g2_subset_df.insert(
    loc=insertion_index,
    column="Withdrawn_by_NCBI",
    value="Value not set"
)

#### Conducting the TSV File Update

In [4]:
from importlib import reload

import NCBI_Entrez_utils as utils

reload(utils)

<module 'NCBI_Entrez_utils' from '/Users/jacobanter/Documents/Code/VACV_screen/Processing_Dharmacon_pooled_genome_1_and_2_subset/NCBI_Entrez_utils.py'>

In [5]:
data_updater = utils.NCBI_Entrez_data_lookup(
    "gene_info_human_9606.tsv",
    "gene_history_human_9606.tsv",
    "gene2accession_human_9606.tsv",
    "gene_refseq_uniprotkb_collab_human_9606.tsv",
    "sec_ac.txt",
    "uniprotkb_organism_id_9606_2025_09_19_all_human_prots_Swiss-Prot_"
    "and_TrEMBL_uniprot_acc_headers.fasta"
)

In [6]:
updated_dp_g1_g2_subset_df = data_updater.check_gene_id_and_symbol(
    dp_g1_g2_subset_df
)

In [7]:
updated_dp_g1_g2_subset_df = data_updater.add_uniprot_ids(
    updated_dp_g1_g2_subset_df
)

In [8]:
updated_dp_g1_g2_subset_df.to_csv(
    "Dharmacon_pooled_G1_G2_screening_plates_subset_updated.tsv",
    sep="\t",
    index=False
)

#### Construction of PPI Pairs for Screen Refinement

In [9]:
# Load the updated TSV file into a Pandas DataFrame
dp_g1_g2_subset_df = pd.read_csv(
    "Dharmacon_pooled_G1_G2_screening_plates_subset_updated.tsv",
    sep="\t"
)

In [10]:
# Extract the UniProt accessions from the Dharmacon pooled subset
# Bear in mind that many entries are composite entries with semicolons
# as separator
# Also remember that many UniProt accessions probably occur multiple
# times, requiring the removal of redundancies via e.g. the Pandas
# `.drop_duplicates()` method
# Yet another layer of complexity stems from the fact that some entries
# are `Nan`, necessitating filtering
dp_accs_list = (
    dp_g1_g2_subset_df["UniProt_IDs"]
    .dropna()
    .str.split(";")
    .explode()
    .drop_duplicates()
    .to_list()
)

In [11]:
print(
    f"There are {len(dp_accs_list):,} UniProtKB protein accessions in "
    "the Dharmacon pooled G1/G2\nscreening plates subset of the screen."
)

There are 43,639 UniProtKB protein accessions in the Dharmacon pooled G1/G2
screening plates subset of the screen.


In [12]:
# Load the FASTA file encompassing all 440 VACV WR proteins
path_to_VACV_WR_fasta = (
    "uniprotkb_organism_id_10254_2025_06_19_all_VACV_WR_prots_uniprot_"
    "only_header.fasta"
)

VACV_WR_fasta = fasta.FastaFile.read(path_to_VACV_WR_fasta)

In [13]:
# Extract the VACV WR UniProt accessions from the FASTA file
VACV_WR_uniprot_accs = list(VACV_WR_fasta.keys())

In [14]:
# Now that both the VACV WR UniProt accessions and the Dharmacon pooled
# subset UniProt accessions have been extracted, they are combined such
# that each and every human protein from the screen is paired with each
# of the 440 VACV WR proteins
# This can be achieved in a very elegant way using a Pandas cross join
# (also called a Cartesian product)

# Convert the lists into DataFrames
human_df = pd.DataFrame({"human_protein": dp_accs_list})
vacv_wr_df = pd.DataFrame({"VACV_WR_protein": VACV_WR_uniprot_accs})

# Perform a cross join
human_vacv_ppi_pairs_df = human_df.merge(
    vacv_wr_df,
    how="cross"
)

In [15]:
human_vacv_ppi_pairs_df.to_csv(
    "Dharmacon_pooled_G1_G2_screening_plates_subset_human-VACV_WR_"
    "PPI_pairs.tsv",
    sep="\t",
    index=False
)

#### Splitting the PPI Pairs TSV File into Chunks

In [7]:
# Load the TSV file harbouring the PPI pairs into a Pandas DataFrame
human_vacv_ppi_pairs_df = pd.read_csv(
    "Dharmacon_pooled_G1_G2_screening_plates_subset_human-VACV_WR_"
    "PPI_pairs.tsv",
    sep="\t"
)

In [8]:
n_ppi_pairs = len(human_vacv_ppi_pairs_df)

print(
    f"Number of human-VACV WR PPI pairs: {n_ppi_pairs:,}"
)

Number of human-VACV WR PPI pairs: 19,201,160


In [9]:
# In total, there are roughly 19,200,000 PPI pairs
# The maximum time for HPC jobs using GPUs is 48 hours, which may be too
# short
# Thus, the PPI pairs are split into chunks each encompassing 5 million
# PPI pairs (6 hours of the 48 hours are reserved for loading the FASTA
# file and the embeddings; the batch size is 64; one batch takes roughly
# 1 second; 42 hours can be used for actual inference;
# 42 hours * 60 * 60 = 151,200 seconds = 151,200 batches;
# 151,200 batches * 64 = 9,676,800 PPI pairs ≈ 9,600,000 PPI pairs;
# thus, up to 9,000,000 PPI pairs per chunk would be possible)
ppi_pair_chunks_dir_path = "PPI_pair_chunks"

if not os.path.exists(ppi_pair_chunks_dir_path):
    os.makedirs(ppi_pair_chunks_dir_path)

In [10]:
# Determine the total number of chunks
CHUNK_SIZE = 5_000_000

n_chunks = math.ceil(n_ppi_pairs / CHUNK_SIZE)

print(f"In total, there are {n_chunks} chunks.")

In total, there are 4 chunks.


In [ ]:
# Now, for each of the 4 chunks, extract the respective subset and save
# it to a separate TSV file
for i in range(n_chunks):
    current_subset = human_vacv_ppi_pairs_df.iloc[
        i * CHUNK_SIZE: (i + 1) * CHUNK_SIZE
    ]

    # Bear in mind that xCAPT5 expects TSV files not to have a header
    current_subset.to_csv(
        os.path.join(
            ppi_pair_chunks_dir_path,
            "Dharmacon_pooled_G1_G2_screening_plates_subset_human-"
            f"VACV_WR_PPI_pairs_chunk_{i}.tsv"
        ),
        sep="\t",
        index=False,
        header=False
    )

#### FASTA File Generation

In [21]:
# Create a FASTA file comprising the 440 VACV WR protein sequences as
# well as the sequences of the human proteins in the Dharmacon pooled
# subset
human_Dharmacon_and_VACV_WR_prots_fasta = fasta.FastaFile()

In [22]:
for header, seq in VACV_WR_fasta.items():
    human_Dharmacon_and_VACV_WR_prots_fasta[header] = seq

In [23]:
# In order to obtain the protein sequences of the human proteins, all
# UniProt Swiss-Prot and UniProt TrEMBL sequences for Homo sapiens (tax
# ID 9606) are downloaded from UniProt
# Only canonical protein sequences are downloaded, i.e. no isoforms
# The download was conducted on September 19th 2025

# Load the FASTA file
all_human_prots_fasta = fasta.FastaFile.read(
    "uniprotkb_organism_id_9606_2025_09_19_all_human_prots_Swiss-Prot_"
    "and_TrEMBL.fasta"
)

In [16]:
# The FASTA file downloaded from UniProt still has default headers
# The headers have to be modified to contain only the UniProt accession
all_human_prots_simple_header_fasta = fasta.FastaFile()

for header, seq in all_human_prots_fasta.items():
    # Conveniently enough, the header elements are separated by pipes/
    # vertical bars with the UniProt accession being the second element
    uniprot_acc = header.split("|")[1]
    all_human_prots_simple_header_fasta[uniprot_acc] = seq

In [17]:
# Save the new FASTA file to disk
all_human_prots_simple_header_fasta.write(
    "uniprotkb_organism_id_9606_2025_09_19_all_human_prots_Swiss-Prot_"
    "and_TrEMBL_uniprot_acc_headers.fasta"
)

In [29]:
# Load the FASTA file with simplified headers
all_human_prots_fasta = fasta.FastaFile.read(
    "uniprotkb_organism_id_9606_2025_09_19_all_human_prots_Swiss-Prot_"
    "and_TrEMBL_uniprot_acc_headers.fasta"
)

In [30]:
# Quickly verify that all human proteins present in the Dharmacon pooled
# subset are also covered by the FASTA file
# Python set lookups are faster
uniprot_accs_in_fasta = set(all_human_prots_fasta.keys())

coverage_list = [
    acc in uniprot_accs_in_fasta
    for acc in dp_accs_list
]

assert all(coverage_list), (
    "Not all human proteins in the Dharmacon pooled subset are covered "
    "by the FASTA file!"
)

In [31]:
# Finally, add the human protein sequences to the FASTA file
for human_prot_acc in dp_accs_list:
    human_Dharmacon_and_VACV_WR_prots_fasta[human_prot_acc] = (
        all_human_prots_fasta[human_prot_acc]
    )

In [32]:
# Perform a sanity check
assert (
    len(human_Dharmacon_and_VACV_WR_prots_fasta)
    ==
    (440 + 43_639)
), "Something went wrong while populating the FASTA file!"

In [33]:
# Now that the sanity check has been successfully passed, save the FASTA
# file to disk
human_Dharmacon_and_VACV_WR_prots_fasta.write(
    "VACV_WR_and_Dharmacon_pooled_G1_G2_screening_plates_human_prots.fasta"
)